<a href="https://colab.research.google.com/github/peppefdf/CSL_Gipuzkoa/blob/main/Isochronic_map_walk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
#%pip install osmnx

import geopandas as gpd
import osmnx as ox
import networkx as nx
import folium
from shapely.geometry import Point


map_center = (43.266120905724044, -1.9759864753467584)
distance = 1000
walk_times = [5,10,15] # in minutes

# download the street network
G = ox.graph_from_point(map_center, dist=distance, network_type='walk', simplify=False)

# get closes graph nodes to origin and destination
orig_node = ox.nearest_nodes(G, map_center[1], map_center[0])

polys = []
for walk_time in walk_times:
        subgraph    = nx.ego_graph(G, orig_node, radius=walk_time, distance='time')
        node_points = [Point(data["x"], data["y"]) for node, data in subgraph.nodes(data=True)]
        polys.append(gpd.GeoSeries(node_points).unary_union.convex_hull)


map = folium.Map(location=[map_center[0], map_center[1]],
                                zoom_start=15,
                                control_scale=True)

style1 = {'fillColor': '#228B22', 'color': '#228B22'}
for i in range(len(walk_times)):
    folium.GeoJson(polys[i], name=str(walk_times[i]), style_function=lambda x:style1).add_to(map)

folium.LayerControl().add_to(map)
map